In [42]:
from tabula import read_pdf
from tabula import convert_into
#Initialize all the libraries needed here
from tabula import read_pdf
from tabula import convert_into
import pandas as pd 
import math
import re
from fuzzywuzzy import fuzz

In [43]:


class ReadCsvFiles:
    def __init__(self,file_path,file_type="csv"):
        if file_type=="csv":
            self.data_df = pd.read_csv(file_path, delimiter='\t')
            self.data_list = self.data_df.values.tolist()
        elif file_type=="xlsx":
            self.data_df = pd.read_excel(file_path)
            self.data_list = self.data_df.values.tolist()
        elif file_type=="pdf":
            self.data_df = read_pdf(file_path, pages="3",multiple_tables=True,area=(400,40,100,100))
    
    def getPdfData(self):
        return self.data_df
    
    def getCsvData(self):
        return self.data_df,self.data_list

In [85]:
members_data,members_list = ReadCsvFiles(r'../Data/Input_Step4/WLP3_(1924_28)_members.csv',"csv").getCsvData()

In [86]:
print("no of rows: ",len(members_list))
total_rows  =len(members_list)

no of rows:  415


In [61]:
#for i in range(total_rows):
#print(members_data)
for member_row in members_list:
    print(member_row[1])

Adams, Otto
Agnes, Lore
Albrecht, Helmuth
Allekotte, Joseph
Andre, Josef
Ansorge, geb. Scholz, Marie
Arendsee, Martha
Arning, Marie
Aufhäuser, Siegfried
Bachmann, Georg
Baecker (Berlin), Paul
Bader, Paul
Bartels, geb. Bieter, Elise
Barth, Georg, Dr. Mr.
Bartschat, Franz
Bauer, Gustav Adolf
Bäumer, Gertrud, vr. Mil.
Bayersdörfer, Michael, vr. niad.
Bazille, Wilhelm
Beck (Oppeln), Adalbert
Becker (Hessen), Johann, Dr. zur.
Becker (Hessen-Nassau), Heinrich
Behm, Margarete, Dr. niecl. k. e.
Behrens, Franz
Bell, Johannes, Dr. Zur.
Bender, Ferdinand
Bergsträßer, Ludwig, Dr. plril.
Berndt, Emil
Bernstein, Eduard
Graf von Bernstorff, Johann-Heinrich
Bertz, Paul
Best, Georg, Dr.
Bensch, Paul, Dr. oee. pudl.
Beythien, Heinrich
Bickes, Theodor
Biener, Franz Hermann
Biester, Louis
Fürst Bismarck, Otto
Blum, Johannes
Bock, Wilhelm
Bockius, Fritz, Dr.
Bohla, Franz
Bohrn-Schuch, Clara
Bolz, Eugen
Bornefeld-ELtmann, Franz
falen, Lippe und Waldcck
Bossen, Peter
Braun (Düsseldorf), Otto
schaften, I)r. z

In [102]:
print(file_1_list[5])

[5, 'Frau Arniilg', 'ja', 'ja']


In [107]:
class MergeFiles:
    def __init__(self):
        self.files_merged =0 
        self.added_rows = 0
        
    #if the main file is the file called first reformat it with headers
    def processMainFile(self,file,file_name):
        #ignoring top 5 lines
        file = file[5:] 
        headers = ["Party","Name"]
        len_val = len(file[0])
        for i in range(2,len_val):
            headers.append(str(file_name+" "+str(i-1)))
        file.insert(0,headers)
        #print("HEADER:",file)
        return file
    
    #add new headers for the additioni file merge and also add dashes for vote columns
    def addAdditionalHeaders(self,file_main_list,file_1_list,file_1_name):
        len_heads = len(file_1_list[5])-2
        for i in range(0,len_heads):
            file_main_list[0].append((file_1_name+" "+str(i+1)))
        #assign dashes by extending the main list 
        for i in range(1,len(file_main_list)):
            for j in range(0,len_heads):
                file_main_list[i].append("-")           
        return file_main_list,len_heads
    #add the matched row to the main file list
    def appendToMainFile(self,file_main_list,matched_id,file_1_row,len_new_cols):
        len_main = len(file_main_list[0])
        j=0
        for i in range(len_main-len_new_cols,len_main):
            file_main_list[matched_id][i] = file_1_row[j]
            j=j+1
        return file_main_list
    
    #create a new row and add the record
    
    def addToMainFile(self,file_main_list,matched_id,file_1_row,len_new_cols):
        len_main = len(file_main_list[0])
        li =[file_1_row[0],file_1_row[1]]
        #print("file row:",file_1_row)
        #li.append()
        for i in range(2,len_main):
            li.append("-")
        j=2
        #print("li 1: ",li)
        for i in range(len_main-len_new_cols,len_main):
            li[i] = file_1_row[j]
            j=j+1
        #print("li 2: ",li)
        file_main_list.append(li)
        return file_main_list
            
    def mergeWithMain(self,file_main_list,file_main_name,file_1_list,file_1_name):
        cnt = 0
        max_matches=0
        if self.files_merged==0:
            file_main_list = self.processMainFile(file_main_list,file_main_name)
        
        self.files_merged = self.files_merged+1
        
        #return file_main_list and no of columns to be added from file_1_list
        file_main_list,len_new_cols = self.addAdditionalHeaders(file_main_list,file_1_list,file_1_name)
        newly_added_row = 0
        
        #return file_main_list
        matches = 0
        print("Total rows in new file: ",len(file_1_list))
        for j in range(len(file_1_list)):
            #Skip 5 lines
            if j<5:
                continue
            #get the name
            str_1 = file_1_list[j][1]
            #get the party name
            party_1 = file_1_list[j][0]
            
            #check  if the name is found in any of it
            did_match =False
            max_percent = 0
            matched_id =-1
            #check if its a string type
            if type(str_1) != str:
                continue
            party_match_percent = 0
            #Iterate main file to search for each name
            for i in range(len(file_main_list)):
                str_main = file_main_list[i][1]
                party_main = file_main_list[i][0]
                
                #check if the name its a string type
                if type(str_main) != str:
                    continue
                    
                #make a  fuzzy check
                percent_match = fuzz.ratio(str_1.lower(),str_main.lower())
                party_match_percent = fuzz.ratio(party_1.lower(),party_main.lower()) 
                
                if percent_match >=max_percent and percent_match>=95 and party_match_percent>=80:
                    #if party_match_percent<=82:
                    #    print("party list: ",party_1," party main:",party_main)
                    #if percent_match <=90:
                    #    print("list main: ",str_1,"----",str_main)
                    max_percent = percent_match
                    matched_id = i
                    #party_match_percent = fuzz.ratio(party_1.lower(),party_main.lower()) 
                    #matches=matches+1
                    #cnt=cnt+1
                    #did_match = True 
                    #break
            #both party and name should match with the main file
            if max_percent>=90:# and party_match_percent>=90 :
                did_match = True
                matches=matches+1
                file_main_list = self.appendToMainFile(file_main_list,matched_id,file_1_list[j][2:],len_new_cols)
            cnt=cnt+1
            #no match then add it to the main file.
            if did_match==False:
                file_main_list = self.addToMainFile(file_main_list,matched_id,file_1_list[j],len_new_cols)
                newly_added_row=newly_added_row+1
                
            max_matches = max(max_matches,matches)
        self.added_rows += newly_added_row
        print("Max Matches per row :",max_matches)
        print("Newly added rows:",newly_added_row)
        #print("Matches:",matches," Count:",cnt," Total rows: ",len(file_1_list))
        return file_main_list
    
    def writeToCSV(self,file_data,name="MergedOutput.csv"):
        df = pd.DataFrame(file_data)
        df.to_csv("../Data/Output_Step4/"+name, sep='\t',header=False,index =False)

In [108]:

from os import listdir
from os.path import isfile, join

mypath = "../Data/Input_Step4/tables/1930_32/"
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]
onlyfiles.sort()
mergeObj = MergeFiles()

file_main_data,file_main_list = ReadCsvFiles('../Data/Input_Step4/tables/1930_32/'+onlyfiles[0],"csv").getCsvData()

for i in range(1,len(onlyfiles)):
    print("Processing .. :",onlyfiles[i])
    file_1_data,file_1_list = ReadCsvFiles('../Data/Input_Step4/tables/1930_32/'+onlyfiles[i],"csv").getCsvData()
    file_main_list = mergeObj.mergeWithMain(file_main_list,onlyfiles[0],file_1_list,onlyfiles[i])
    if i==445:
        print(onlyfiles[4])
        break
#mergeObj.writeToCSV(file_main_data)

Processing .. : 1930 32,1.pdf_222_225.csv
Total rows in new file:  546
Max Matches per row : 373
Newly added rows: 168
Processing .. : 1930 32,1.pdf_448_454.csv
Total rows in new file:  560
Max Matches per row : 448
Newly added rows: 107
Processing .. : 1930 32,1.pdf_493_504.csv
Total rows in new file:  411
Max Matches per row : 354
Newly added rows: 52
Processing .. : 1930 32,1.pdf_595_598.csv
Total rows in new file:  542
Max Matches per row : 446
Newly added rows: 91
Processing .. : 1930 32,1.pdf_641_644.csv
Total rows in new file:  546
Max Matches per row : 497
Newly added rows: 44
Processing .. : 1930 32,1.pdf_659_663.csv
Total rows in new file:  527
Max Matches per row : 432
Newly added rows: 90
Processing .. : 1930 32,1.pdf_775_786.csv
Total rows in new file:  481
Max Matches per row : 368
Newly added rows: 108
Processing .. : 1930 32,1.pdf_864_867.csv
Total rows in new file:  393
Max Matches per row : 333
Newly added rows: 55
Processing .. : 1930 32,1.pdf_951_954.csv
Total rows 

In [109]:
mergeObj.added_rows

1576

In [106]:
new_file=sorted(file_main_list, key=lambda x: (x[0], x[1]))
new_file.reverse()
mergeObj.writeToCSV(new_file,name="1930_32_merge_sorted_final.csv")


In [96]:
file_main_name =  "1924:28,8_102_105.csv" #""
file_main_data,file_main_list = ReadCsvFiles('../Data/Input_Step4/tables/'+file_main_name,"csv").getCsvData()

In [97]:
file_1_name = "1924:28,1_371_374.csv"
file_1_data,file_1_list = ReadCsvFiles('../Data/Input_Step4/tables/'+file_1_name,"csv").getCsvData()

In [98]:
mergeObj = MergeFiles()
merged_out = mergeObj.mergeWithMain(file_main_list,file_main_name,file_1_list,file_1_name)
mergeObj.writeToCSV(merged_out)

file row: ['-', 'Frau Agnes', '-']
li 1:  ['-', 'Frau Agnes', '-', '-', '-']
li 2:  ['-', 'Frau Agnes', '-', '-', '-']
file row: ['-', 'Frau Arning', 'za']
li 1:  ['-', 'Frau Arning', '-', '-', '-']
li 2:  ['-', 'Frau Arning', '-', '-', 'za']
file row: ['-', 'Wachmann', 'nein']
li 1:  ['-', 'Wachmann', '-', '-', '-']
li 2:  ['-', 'Wachmann', '-', '-', 'nein']
file row: ['-', 'Baecker (Berlin)', '-']
li 1:  ['-', 'Baecker (Berlin)', '-', '-', '-']
li 2:  ['-', 'Baecker (Berlin)', '-', '-', '-']
file row: ['-', 'Frau Bartels', 'za']
li 1:  ['-', 'Frau Bartels', '-', '-', '-']
li 2:  ['-', 'Frau Bartels', '-', '-', 'za']
file row: ['-', 'Bauer', 'za']
li 1:  ['-', 'Bauer', '-', '-', '-']
li 2:  ['-', 'Bauer', '-', '-', 'za']
file row: ['-', 'vr. Bayersdörfer', '-']
li 1:  ['-', 'vr. Bayersdörfer', '-', '-', '-']
li 2:  ['-', 'vr. Bayersdörfer', '-', '-', '-']
file row: ['-', 'Becker (Hessen-Nassau)', 'za']
li 1:  ['-', 'Becker (Hessen-Nassau)', '-', '-', '-']
li 2:  ['-', 'Becker (Hessen-

file row: ['-', 'Frau Neuhaus (West- falen)', '-']
li 1:  ['-', 'Frau Neuhaus (West- falen)', '-', '-', '-']
li 2:  ['-', 'Frau Neuhaus (West- falen)', '-', '-', '-']
file row: ['-', 'Steyses', 'nein']
li 1:  ['-', 'Steyses', '-', '-', '-']
li 2:  ['-', 'Steyses', '-', '-', 'nein']
file row: ['-', 'Nientimp', '-']
li 1:  ['-', 'Nientimp', '-', '-', '-']
li 2:  ['-', 'Nientimp', '-', '-', '-']
file row: ['-', 'Passehl', 'ja']
li 1:  ['-', 'Passehl', '-', '-', '-']
li 2:  ['-', 'Passehl', '-', '-', 'ja']
file row: ['-', 'Du Pfleger', 'nein']
li 1:  ['-', 'Du Pfleger', '-', '-', '-']
li 2:  ['-', 'Du Pfleger', '-', '-', 'nein']
file row: ['-', 'Dr. Philipp', 'nein']
li 1:  ['-', 'Dr. Philipp', '-', '-', '-']
li 2:  ['-', 'Dr. Philipp', '-', '-', 'nein']
file row: ['-', 'Rami»', 'ja']
li 1:  ['-', 'Rami»', '-', '-', '-']
li 2:  ['-', 'Rami»', '-', '-', 'ja']
file row: ['-', 'Dr. Reichert', 'nein']
li 1:  ['-', 'Dr. Reichert', '-', '-', '-']
li 2:  ['-', 'Dr. Reichert', '-', '-', 'nein']
fi

In [101]:
file_1_name = "1928:30,1_128_131.csv"#"1924:28,1_241_244.csv"
file_1_data,file_1_list = ReadCsvFiles('../Data/Input_Step4/tables/'+file_1_name,"csv").getCsvData()
merged_out = mergeObj.mergeWithMain(merged_out,file_main_name,file_1_list,file_1_name)

file row: ['sozias demokratische partei', 'Arzt', 'nein', 'za']
li 1:  ['sozias demokratische partei', 'Arzt', '-', '-', '-', '-', '-', '-']
li 2:  ['sozias demokratische partei', 'Arzt', '-', '-', '-', '-', 'nein', 'za']
file row: ['sozias demokratische partei', 'Beims', 'krank', 'krank']
li 1:  ['sozias demokratische partei', 'Beims', '-', '-', '-', '-', '-', '-']
li 2:  ['sozias demokratische partei', 'Beims', '-', '-', '-', '-', 'krank', 'krank']
file row: ['sozias demokratische partei', 'Bergmann', 'nein', 'ja']
li 1:  ['sozias demokratische partei', 'Bergmann', '-', '-', '-', '-', '-', '-']
li 2:  ['sozias demokratische partei', 'Bergmann', '-', '-', '-', '-', 'nein', 'ja']
file row: ['sozias demokratische partei', 'Böckler', 'nein', 'ja']
li 1:  ['sozias demokratische partei', 'Böckler', '-', '-', '-', '-', '-', '-']
li 2:  ['sozias demokratische partei', 'Böckler', '-', '-', '-', '-', 'nein', 'ja']
file row: ['sozias demokratische partei', 'Brandes', 'nein', 'za']
li 1:  ['sozi

file row: ['sozias demokratische partei', 'Gottheiner', 'za', 'nein']
li 1:  ['sozias demokratische partei', 'Gottheiner', '-', '-', '-', '-', '-', '-']
li 2:  ['sozias demokratische partei', 'Gottheiner', '-', '-', '-', '-', 'za', 'nein']
file row: ['sozias demokratische partei', 'Hartwig (Berlin)', 'za', 'nein']
li 1:  ['sozias demokratische partei', 'Hartwig (Berlin)', '-', '-', '-', '-', '-', '-']
li 2:  ['sozias demokratische partei', 'Hartwig (Berlin)', '-', '-', '-', '-', 'za', 'nein']
file row: ['sozias demokratische partei', 'Dr. Haßlacher', 'za', 'nein']
li 1:  ['sozias demokratische partei', 'Dr. Haßlacher', '-', '-', '-', '-', '-', '-']
li 2:  ['sozias demokratische partei', 'Dr. Haßlacher', '-', '-', '-', '-', 'za', 'nein']
file row: ['sozias demokratische partei', 'Panibach', 'za', 'nein']
li 1:  ['sozias demokratische partei', 'Panibach', '-', '-', '-', '-', '-', '-']
li 2:  ['sozias demokratische partei', 'Panibach', '-', '-', '-', '-', 'za', 'nein']
file row: ['sozias 

file row: ['kommu nistische partei', 'Graf (Dresden)', '3a', 'nein']
li 1:  ['kommu nistische partei', 'Graf (Dresden)', '-', '-', '-', '-', '-', '-']
li 2:  ['kommu nistische partei', 'Graf (Dresden)', '-', '-', '-', '-', '3a', 'nein']
file row: ['kommu nistische partei', 'Hein', '3a', 'nein']
li 1:  ['kommu nistische partei', 'Hein', '-', '-', '-', '-', '-', '-']
li 2:  ['kommu nistische partei', 'Hein', '-', '-', '-', '-', '3a', 'nein']
file row: ['kommu nistische partei', 'Kaßler', '3«', 'nein']
li 1:  ['kommu nistische partei', 'Kaßler', '-', '-', '-', '-', '-', '-']
li 2:  ['kommu nistische partei', 'Kaßler', '-', '-', '-', '-', '3«', 'nein']
file row: ['kommu nistische partei', 'Kippenberqer', '3«', 'nein']
li 1:  ['kommu nistische partei', 'Kippenberqer', '-', '-', '-', '-', '-', '-']
li 2:  ['kommu nistische partei', 'Kippenberqer', '-', '-', '-', '-', '3«', 'nein']
file row: ['kommu nistische partei', 'Kollwik', '3a', 'nein']
li 1:  ['kommu nistische partei', 'Kollwik', '-', 

file row: ['bayerische bolkspartei', 'Schmitt (Franken)', 'nein', '3«']
li 1:  ['bayerische bolkspartei', 'Schmitt (Franken)', '-', '-', '-', '-', '-', '-']
li 2:  ['bayerische bolkspartei', 'Schmitt (Franken)', '-', '-', '-', '-', 'nein', '3«']
file row: ['nat.-soz. dtsch. arbeiter-partei', 'Buch Dreher', 'ja', 'nein']
li 1:  ['nat.-soz. dtsch. arbeiter-partei', 'Buch Dreher', '-', '-', '-', '-', '-', '-']
li 2:  ['nat.-soz. dtsch. arbeiter-partei', 'Buch Dreher', '-', '-', '-', '-', 'ja', 'nein']
file row: ['nat.-soz. dtsch. arbeiter-partei', 'Ritter Epp', 'ja', 'nein']
li 1:  ['nat.-soz. dtsch. arbeiter-partei', 'Ritter Epp', '-', '-', '-', '-', '-', '-']
li 2:  ['nat.-soz. dtsch. arbeiter-partei', 'Ritter Epp', '-', '-', '-', '-', 'ja', 'nein']
file row: ['nat.-soz. dtsch. arbeiter-partei', 'Jeder (Sachsen) Dr. Frick', 'za', 'nein']
li 1:  ['nat.-soz. dtsch. arbeiter-partei', 'Jeder (Sachsen) Dr. Frick', '-', '-', '-', '-', '-', '-']
li 2:  ['nat.-soz. dtsch. arbeiter-partei', 'Jed

In [102]:
mergeObj.writeToCSV(merged_out)